In [6]:
!wget "https://raw.githubusercontent.com/berpj/datasets/master/speeches/elon_musk_speeches.txt"  

--2023-06-12 15:45:43--  https://raw.githubusercontent.com/berpj/datasets/master/speeches/elon_musk_speeches.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 665131 (650K) [text/plain]
Saving to: ‘elon_musk_speeches.txt’

elon_musk_speeches. 100%[===================>] 649.54K  --.-KB/s    in 0.01s   

2023-06-12 15:45:44 (50.5 MB/s) - ‘elon_musk_speeches.txt’ saved [665131/665131]



In [22]:
# import pandas as pd 
# df = pd.read_table("elon_musk_speeches.txt", header=None)
# df = df.reset_index()
# df = df.rename(columns={0: "speech"})[["speech"]]
# df["speaker"] = "Elon Musk"
# df.head()

,speech,speaker
0,I'll try to make this as interesting as possib...,Elon Musk
1,So we had that but I wanted to do something mo...,Elon Musk
2,And that's just an approximate evolution of th...,Elon Musk
3,"In about February of last year, I'm sure you'r...",Elon Musk
4,"It happened coincidentally, that in the first ...",Elon Musk


In [23]:
# df.to_csv("data/speeches/elon-musk.csv", index=False)

In [1]:
!nvidia-smi

Mon Jun 12 15:32:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   39C    P8    24W / 300W |  37221MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:41:00.0 Off |                  Off |
| 30%   

In [2]:
import os 
os.environ["HF_HOME"] ="/fs/nexus-scratch/skarki/.cache/huggingface"

In [5]:
import pandas as pd 

df = pd.read_csv("~/personalized-llm/data/speeches/thomas-jefferson.csv")

df.head()

,speech,speaker
0,"FRIENDS AND FELLOW-CITIZENS, Called upon to...",Thomas Jefferson
1,"GENTLEMAN, I have received the remonstrance ...",Thomas Jefferson
2,FELLOW CITIZENS OF THE SENATE AND HOUSE OF RE...,Thomas Jefferson
3,"GENTLEMAN, The affectionate sentiments of es...",Thomas Jefferson
4,TO BROTHER HANDSOME LAKE: I h...,Thomas Jefferson


In [6]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

load_8bit = True
device_map = "auto"
base_model = "WizardLM/WizardLM-13B-V1.0"
tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=load_8bit,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

Loading checkpoint shards: 100%|██████████| 6/6 [03:28<00:00, 34.68s/it]


NameError: name 'sys' is not defined

In [7]:
import torch
import sys
if torch.__version__ >= "2" and sys.platform != "win32":
    print("Compiling....")
    model = torch.compile(model)

Compiling....


In [15]:
from transformers import GenerationConfig

device = "cuda" 
def inference(
            batch_data,
            input=None,
            temperature=1,
            top_p=0.95,
            top_k=10,
            num_beams=1,
            max_new_tokens=2048,
            **kwargs,
    ):
        
        
        prompts = f"""A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {batch_data} ASSISTANT:"""
        
        inputs = tokenizer(prompts, return_tensors="pt")
        input_ids = inputs["input_ids"].to(device)
        generation_config = GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_beams=num_beams,
            **kwargs,
        )
        with torch.no_grad():
            generation_output = model.generate(
                input_ids=input_ids,
                generation_config=generation_config,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
            )
        s = generation_output.sequences       
        output = tokenizer.batch_decode(s, skip_special_tokens=True)
        output = output[0].split("ASSISTANT:")[1].strip()
       
        return output

In [16]:


# instruction = """Instruction: Predict the speaker of the following speech based on the style and not the content of the speech. Only choose speakers from among the given choices. Also give your confidence score between 0 and 1 (in percentage) for the prediction and a brief explanation of why you think so.

# Choices: Barack Obama, Donald Trump, Joe Biden, Thomas Jefferson, Abraham Lincoln, Lyndon B. Johnson, Ronald Reagan, Franklin D. Roosevelt, George W. Bush, Bill Clinton, Woodrow Wilson

# Speech:  Well, the poll numbers will have to agree because it's... we blow everybody away. And that's because we did a great job. We had a great four years. Then we had a terrible election, a terrible, terrible thing happened. It was a rigged election. And we'll now see what happens. But that was just a terrible thing happened to our country. Because our country... It's suffered so badly over the last two years. When you look at inflation, when you look at Afghanistan, when you look at all of the things that have happened that have been so bad, and we were doing so well before that. So, we'll make a decision very soon.
# """
instruction = "What is the area of a rectangle with length 12 cm and width 8 cm? A) 48 cm^2 B) 96 cm^2 C) 120 cm^2 D) 192 cm^2"
final_output = inference(instruction)
final_output

'Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats charact zus Probably weiteren��志Stats chara

In [30]:
prompt = """Predict the speaker of the following speech based on the style of the speech and not the content of the speech. Choose the speaker from among the choices.
 
Speech: Well, the poll numbers will have to agree because it's... we blow everybody away. And that's because we did a great job. We had a great four years. Then we had a terrible election, a terrible, terrible thing happened. It was a rigged election. And we'll now see what happens. But that was just a terrible thing happened to our country. Because our country... It's suffered so badly over the last two years. When you look at inflation, when you look at Afghanistan, when you look at all of the things that have happened that have been so bad, and we were doing so well before that. So, we'll make a decision very soon.
Choices: Donald Trump, Barack Obama, Joe Biden
Speaker: Donald Trump

Speech:  I think our adversaries have seen us weakened, not just as a consequence of this election, but over the last several years. We have these cleavages in the body politic that they’re convinced they can exploit. There is an old adage that partisan politics should stop at the water’s edge. That when it comes to our foreign policy, that it is the United States of America, not the divided States of America.
Choices: Donald Trump, Barack Obama, Joe Biden
Speaker: Barack Obama

Speech: And by virtue of the power, and for the purpose aforesaid, I do order and declare that all persons held as slaves within said designated States, and parts of States, are, and henceforward shall be free; and that the Executive government of the United States, including the military and naval authorities thereof, will recognize and maintain the freedom of said persons.
Choices: Donald Trump, Barack Obama, Joe Biden
Speaker: 
"""

inputs = tokenizer(prompt, return_tensors="pt", max_length=3600).to("cuda")

out = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=128, temperature=0.8, top_p=0.8, repetition_penalty=1.2)
tokenizer.decode(out[0], skip_special_tokens=True)

'Predict the speaker of the following speech based on the style of the speech and not the content of the speech. Choose the speaker from among the choices.\n \nSpeech: Well, the poll numbers will have to agree because it\'s... we blow everybody away. And that\'s because we did a great job. We had a great four years. Then we had a terrible election, a terrible, terrible thing happened. It was a rigged election. And we\'ll now see what happens. But that was just a terrible thing happened to our country. Because our country... It\'s suffered so badly over the last two years. When you look at inflation, when you look at Afghanistan, when you look at all of the things that have happened that have been so bad, and we were doing so well before that. So, we\'ll make a decision very soon.\nChoices: Donald Trump, Barack Obama, Joe Biden\nSpeaker: Donald Trump\n\nSpeech:  I think our adversaries have seen us weakened, not just as a consequence of this election, but over the last several years. We

In [ ]:
# prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 
# ### Instruction: Predict the speaker of the following speech based on the style and not the content of the speech. 
# ### Input:  
# Speech: Well, the poll numbers will have to agree because it's... we blow everybody away. And that's because we did a great job. We had a great four years. Then we had a terrible election, a terrible, terrible thing happened. It was a rigged election. And we'll now see what happens. But that was just a terrible thing happened to our country. Because our country... It's suffered so badly over the last two years. When you look at inflation, when you look at Afghanistan, when you look at all of the things that have happened that have been so bad, and we were doing so well before that. So, we'll make a decision very soon.
# Choices: Joe Biden, Donald Trump, Barack Obama
# Speaker: Donald Trump

# Speech:  I think our adversaries have seen us weakened, not just as a consequence of this election, but over the last several years. We have these cleavages in the body politic that they’re convinced they can exploit. There is an old adage that partisan politics should stop at the water’s edge. That when it comes to our foreign policy, that it is the United States of America, not the divided States of America.
# Choices: Joe Biden, Donald Trump, Barack Obama
# Speaker: Barack Obama

# Speech: Look, I don't think it was a fa-- look, it was a simple choice, George. When the-- when the Taliban -- let me back -- put it another way. When you had the government of Afghanistan, the leader of that government get in a plane and taking off and going to another country, when you saw the significant collapse of the ta-- of the-- Afghan troops we had trained -- up to 300,000 of them just leaving their equipment and taking off, that was -- you know, I'm not-- this -- that -- that's what happened.
# Choices: Joe Biden, Donald Trump, Barack Obama
# Speaker: """

# inputs = tokenizer(prompt, return_tensors="pt", max_length=2048).to("cuda")

# out = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=128, temperature=0.8, top_p=0.8, repetition_penalty=1.2)
# tokenizer.decode(out[0], skip_special_tokens=True)

In [34]:
out = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, max_length=128, temperature=0.9, top_p=0.8, repetition_penalty=1.3)
tokenizer.decode(out[0], skip_special_tokens=True)

'### Instruction: Identify the sentiment in the following text. Choose one of positive or negative.\n\n### Input: I hate this dish!\n\n### Sentiment: \n    # -1 if it is a negative sentence, otherwise +1 (0 means neutral)\n    2 * np.log(len([w for w in re.findall(\'[a-zA-Z]+\', \'I hate\')]) / len([\'hate\' in word] & [word != "this" and not word == ""]))'